In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
import psutil
import os
import time
import sys
import shutil
%matplotlib inline

In [23]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [24]:
src_path = df['.dat path'].iloc[-1]
dst_dir = '/datax/scratch/jliang/'
dst_path = os.path.join(dst_dir, 'high_data_dat')  # Keeps original filename

shutil.copy(src_path, dst_path)

'/datax/scratch/jliang/high_data_dat'

In [16]:
chunk_size = 3000
group_size = 6
n_rows = len(df)

for batch_start in range(n_rows - 1, n_rows, group_size):
    batch_end = min(batch_start + group_size, n_rows)
    available = psutil.virtual_memory()[1]
    print(f"Processing rows {batch_start}-{batch_end-1}, available memory: {available/1e9:.2f} GB")

    batch_arrays = []
    for idx in range(batch_start, batch_end):
        # load the waterfall and get its frequency axis
        fb = bl.Waterfall(df['.h5 path'].iloc[idx])
        # header = fb.header
        # n_freq = header['nchans']
        # fch1 = header['fch1']     # freq of channel 0 in MHz
        # foff = header['foff']     # channel spacing in MHz (often negative)
        
        # # build the full freq array and grab its min/max
        # freqs = fch1 + np.arange(n_freq) * foff
        # f_min, f_max = freqs.min(), freqs.max()

        # # read hits table
        # dat = df['.dat path'].iloc[idx]
        # dat_df = pd.read_table(dat, sep='\s+', names=[
        #         'Top_Hit_#','Drift_Rate','SNR','Uncorrected_Frequency',
        #         'Corrected_Frequency','Index','freq_start','freq_end',
        #         'SEFD','SEFD_freq','Coarse_Channel_Number','Full_number_of_hits'
        #     ],
        #     skiprows=9
        # )

        # # set the RFI label if any hit overlaps the band
        # overlap = ((dat_df['freq_start'] <= f_max) & (dat_df['freq_end'] >= f_min))
        # df.at[idx, 'RFI_label'] = overlap.any()

        data = np.squeeze(fb.data) # shape = (time, freq)
        n_chunks = data.shape[1] // chunk_size
        print('Data shape:', data.shape, 'n_chunks:', n_chunks)

        # optional memory check
        est_bytes = n_chunks * data.shape[0] * chunk_size * data.dtype.itemsize
        if est_bytes > available * 0.9:
            raise MemoryError(
                f"Row {idx} needs ~{est_bytes/1e9:.2f} GB but only {available/1e9:.2f} GB free"
            )

        arr = np.array([
            data[:, j*chunk_size:(j+1)*chunk_size]
            for j in range(n_chunks)
        ])  # shape = (n_chunks, freq, chunk_size)
        batch_arrays.append(arr)
        print("Array shape:", arr.shape)

    # ensure all batch_arrays have the same second‐dimension before stacking 
    times = [a.shape[1] for a in batch_arrays]
    min_times = min(times)
    if len(set(times)) > 1:
        print(f"Trimming each array from {times} chunks down to {min_times} chunks")
        batch_arrays = [a[:, :min_times, :] for a in batch_arrays]

    # final memory check for the whole batch
    total_bytes = sum(a.nbytes for a in batch_arrays)
    available = psutil.virtual_memory()[1]
    if total_bytes > available * 0.9:
        raise MemoryError(
            f"Batch {batch_start}-{batch_end-1} needs ~{total_bytes/1e9:.2f} GB but only {available/1e9:.2f} GB free"
        )

    batch_array = np.stack(batch_arrays, axis=0)
    out_path    = f'/datax/scratch/jliang/high_data_array_rows_{batch_start}_{batch_end}.npy'
    np.save(out_path, batch_array)
    print(f"Saved rows {batch_start}-{batch_end-1} -> {out_path}")

Processing rows 36557-36557, available memory: 83.50 GB
Data shape: (16, 67108864) n_chunks: 22369
Array shape: (22369, 16, 3000)
Saved rows 36557-36557 -> /datax/scratch/jliang/high_data_array_rows_36557_36558.npy


In [17]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
#df['.dat path'] = df['.dat path'].str.replace('0000.dat', '0002.dat', regex=False)
print(df['.h5 path'].iloc[-1])
print(df['.dat path'].iloc[-1])

/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc16_guppi_59902_24005_HIP50744_0057.rawspec.0002.h5
/home/obs/turboseti/AGBT22B_999_25/blc16_blp06/blc16_guppi_59902_24005_HIP50744_0057.rawspec.0000/blc16_guppi_59902_24005_HIP50744_0057.rawspec.0000.dat


In [18]:
# dat = df['.dat path'].iloc[-1]
# print(dat)

# dat_df = pd.read_table(dat, sep='\s+', names=['Top_Hit_#','Drift_Rate','SNR','Uncorrected_Frequency','Corrected_Frequency',
#                                                 'Index', 'freq_start', 'freq_end', 'SEFD', 'SEFD_freq', 'Coarse_Channel_Number', 
#                                                 'Full_number_of_hits'], skiprows=9)
# dat_df.iloc[10]

In [19]:
# snr = 10
# outdir  = '/datax/scratch/jliang/'

# # threshold of 20 GB in bytes
# threshold = 20 * (1024 ** 3)
# count = 0

# for i, file in df['.h5 path'].items():
#     while psutil.virtual_memory().available < threshold:
#         print('Waiting for memory to free up…')
#         time.sleep(60)

#     # build a base name without its extension
#     base = os.path.splitext(os.path.basename(file))[0]
#     output_path = os.path.join(outdir, f"{base}_{snr}_seticore.dat")
#     fb = bl.Waterfall(file, load_data=False)
#     hdr = fb.header                # contains 'nchans'
#     nch = hdr['nchans']
#     if 'spliced' in file:
#         nfpc = nch
#     else:
#         nfpc = nch / 64

#     # run seticore
#     console = 'seticore ' + file + ' -M 4 -s ' + str(snr) + ' --fine_channels ' + str(int(nfpc)) + ' --output ' + outdir + os.path.basename(file)[:-2] + str(snr) + '_seticore.dat'
#     exit_code = os.system(console)
#     if exit_code != 0:
#         print(f" Error: `seticore` returned non-zero exit code {exit_code} for file {file}")
#         sys.exit(exit_code)

#     # safely assign into the DataFrame
#     df.loc[i, '.dat path'] = output_path
#     print("Wrote:", output_path)

#     dat_df = pd.read_table(output_path, sep='\s+', names=['Top_Hit_#','Drift_Rate','SNR','Uncorrected_Frequency','Corrected_Frequency',
#                                                 'Index', 'freq_start', 'freq_end', 'SEFD', 'SEFD_freq', 'Coarse_Channel_Number', 
#                                                 'Full_number_of_hits'], skiprows=9)
#     df['Corrected_Frequency'] = dat_df['Corrected_Frequency']
#     df['Drift_Rate'] = dat_df['Drift_Rate']
#     df['SNR'] = dat_df['SNR']
#     count += 1
#     if count == 1000:
#         break

# # finally, save your CSV
# df.to_csv('/datax/scratch/jliang/galaxies_cadences_small.csv', index=False)

In [20]:
#df['RFI_label'] = False
chunk_size = 3000
group_size = 6
n_rows = len(df)

for batch_start in range(n_rows - 1, n_rows, group_size):
    batch_end = min(batch_start + group_size, n_rows)
    available = psutil.virtual_memory()[1]
    print(f"Processing rows {batch_start}-{batch_end-1}, available memory: {available/1e9:.2f} GB")

    batch_arrays = []
    for idx in range(batch_start, batch_end):
        # load the waterfall and get its frequency axis
        fb = bl.Waterfall(df['.h5 path'].iloc[idx])
        # header = fb.header
        # n_freq = header['nchans']
        # fch1 = header['fch1']     # freq of channel 0 in MHz
        # foff = header['foff']     # channel spacing in MHz (often negative)
        
        # # build the full freq array and grab its min/max
        # freqs = fch1 + np.arange(n_freq) * foff
        # f_min, f_max = freqs.min(), freqs.max()

        # # read hits table
        # dat = df['.dat path'].iloc[idx]
        # dat_df = pd.read_table(dat, sep='\s+', names=[
        #         'Top_Hit_#','Drift_Rate','SNR','Uncorrected_Frequency',
        #         'Corrected_Frequency','Index','freq_start','freq_end',
        #         'SEFD','SEFD_freq','Coarse_Channel_Number','Full_number_of_hits'
        #     ],
        #     skiprows=9
        # )

        # # set the RFI label if any hit overlaps the band
        # overlap = ((dat_df['freq_start'] <= f_max) & (dat_df['freq_end'] >= f_min))
        # df.at[idx, 'RFI_label'] = overlap.any()

        data = np.squeeze(fb.data) # shape = (time, freq)
        n_chunks = data.shape[1] // chunk_size
        print('Data shape:', data.shape, 'n_chunks:', n_chunks)

        # optional memory check
        est_bytes = n_chunks * data.shape[0] * chunk_size * data.dtype.itemsize
        if est_bytes > available * 0.9:
            raise MemoryError(
                f"Row {idx} needs ~{est_bytes/1e9:.2f} GB but only {available/1e9:.2f} GB free"
            )

        arr = np.array([
            data[:, j*chunk_size:(j+1)*chunk_size]
            for j in range(n_chunks)
        ])  # shape = (n_chunks, freq, chunk_size)
        batch_arrays.append(arr)
        print("Array shape:", arr.shape)

    # ensure all batch_arrays have the same second‐dimension before stacking 
    times = [a.shape[1] for a in batch_arrays]
    min_times = min(times)
    if len(set(times)) > 1:
        print(f"Trimming each array from {times} chunks down to {min_times} chunks")
        batch_arrays = [a[:, :min_times, :] for a in batch_arrays]

    # final memory check for the whole batch
    total_bytes = sum(a.nbytes for a in batch_arrays)
    available = psutil.virtual_memory()[1]
    if total_bytes > available * 0.9:
        raise MemoryError(
            f"Batch {batch_start}-{batch_end-1} needs ~{total_bytes/1e9:.2f} GB but only {available/1e9:.2f} GB free"
        )

    batch_array = np.stack(batch_arrays, axis=0)
    out_path    = f'/datax/scratch/jliang/mid_data_array_rows_{batch_start}_{batch_end}.npy'
    np.save(out_path, batch_array)
    print(f"Saved rows {batch_start}-{batch_end-1} -> {out_path}")

Processing rows 36557-36557, available memory: 88.28 GB
Data shape: (279, 65536) n_chunks: 21
Array shape: (21, 279, 3000)
Saved rows 36557-36557 -> /datax/scratch/jliang/mid_data_array_rows_36557_36558.npy


In [21]:
print(batch_array.shape)
print(batch_array.dtype)

(1, 21, 279, 3000)
float32


In [5]:
file = df['.h5 path'].values[36552]
print(file)

fb = bl.Waterfall(file)
fb.info()

/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc16_guppi_59902_22416_NGC3226_0052.rawspec.0002.h5

--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :                1126.46484375 MHz
            foff :         -0.00286102294921875 MHz
           ibeam :                                1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                            65536
            nfpc :                             1024
            nifs :                                1
     rawdatafile : guppi_59902_22416_NGC3226_0052.0000.raw
     source_name :                          NGC3226
         src_dej :                      19:53:54.96
         src_raj :                     10:23:26.952
    telescope_id :                           

In [8]:
fb.info()
fb_data = fb.data
print(fb_data.shape)


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :            8438.963413238525 MHz
            foff :         -0.00286102294921875 MHz
      machine_id :                               20
           nbits :                               32
          nchans :                          1703936
            nifs :                                1
     source_name :                       Bol520_off
         src_dej :                      34:54:57.96
         src_raj :                        0:50:42.6
    telescope_id :                                6
           tsamp :               1.0737418239999998
   tstart (ISOT) :          2018-03-08T16:41:30.000
    tstart (MJD) :                58185.69548611111
        za_start :                              0.0

Num ints in file :                              547
      File shape :                (547, 1, 